In [1]:
%pip install mglearn
%pip install cvxpy

  Using cached mglearn-0.2.0-py2.py3-none-any.whl.metadata (628 bytes)
Using cached mglearn-0.2.0-py2.py3-none-any.whl (581 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached cvxpy-1.6.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.3 kB)
  Using cached osqp-1.0.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached clarabel-0.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.8 kB)
  Using cached scs-3.2.7.post2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
Using cached cvxpy-1.6.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
Using cached clarabel-0.10.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)
Using cached osqp-1.0.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (343 kB)
Using cached scs-3.2.7.post2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (10.4 MB)
Note: y

In [2]:
#Libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import pandas as pd
import cvxpy as cp
import mglearn
import math
from sklearn.preprocessing import MinMaxScaler

In [3]:
#read data into a var and print column names
#data: https://www.kaggle.com/datasets/programmer3/real-time-network-traffic-encryption-dataset/data
enc_data = pd.read_csv('traffic_encryption_data.csv')#full set
voip_data = pd.read_csv('Voip_data.csv')
vid_data = pd.read_csv('Video_data.csv')
cloud_data = pd.read_csv('Cloud_data.csv')
print(enc_data.columns)

Index(['packet_size', 'transmission_time', 'latency', 'jitter', 'packet_loss',
       'network_load', 'throughput', 'bandwidth_usage', 'encryption_time',
       'decryption_time', 'key_size', 'block_size', 'cpu_usage',
       'memory_usage', 'encryption_strength', 'error_rate',
       'integrity_check_time', 'retransmission_count', 'packet_order',
       'traffic_type'],
      dtype='object')


In [4]:
#selected columns raw and to be normalized
selected_norm_cols = ['encryption_time','encryption_strength','error_rate','integrity_check_time','cpu_usage','memory_usage']

selected_raw_cols = ['packet_size','latency','jitter','packet_loss','throughput']

In [5]:
#diagonal Q matrix function
#err unreliable output more stable dependable configurations
#integ defends against tampering and file attacks
def Q_matrix_const(times, strengths, err, integ, lam_1=1.0, lam_2=1.0, lam_3=1.0):
    Q_diag = times + lam_1 * (1 - strengths) + lam_2 * err - lam_3 * integ
    Q_diag = np.maximum(Q_diag, 0) 
    Q_matrix = np.diag(Q_diag)
    return Q_matrix

In [6]:
#gamma weight of resource cost
def resource_cost(cpu, mem, w, gamma=1.0):
    c = cpu + mem
    return gamma * c @ w

In [7]:
#size categories function
def size_categories(df, size_col = 'packet_size'):
    small = 500
    medium = 1000
    return {
            'Small': df[df[size_col] <= small],
            'Medium': df[(df[size_col] > small) & (df[size_col] <= medium)],
            'Large': df[df[size_col] > medium]
        }


In [22]:
def solve_and_report(enc_data, data_type=None, alpha=1.0, gamma=1.0, lam_1=1.0, lam_2=1.0, lam_3=1.0):
    #packet size categories
    packet_groups = size_categories(enc_data, size_col='packet_size') 

    #solve and print
    for group_name, group_df in packet_groups.items():
        print(f"\nSolving for Packet Size Group: {group_name}")

        #normalize only selected features and keep raw ones
        norm_subset = group_df[selected_norm_cols]
        scaler = MinMaxScaler()
        normalized_part = pd.DataFrame(scaler.fit_transform(norm_subset), columns=selected_norm_cols)
        raw_subset = group_df[list(selected_raw_cols)].reset_index(drop=True)
        normalized = pd.concat([normalized_part, raw_subset], axis=1)

        #extract normalized data
        enc_times = normalized['encryption_time'].values
        enc_strengths = normalized['encryption_strength'].values
        err_rates = normalized['error_rate'].values 
        integ_times = normalized['integrity_check_time'].values
        cpu_usages = normalized['cpu_usage'].values
        mem_usages = normalized['memory_usage'].values

        num_scenarios = len(normalized)
        if num_scenarios == 0:
            print(f"No data in {group_name} category.")
            continue

        #decision variables
        w = cp.Variable(num_scenarios)
        
        #equation generation
        Q = Q_matrix_const(enc_times, enc_strengths, err_rates, integ_times, lam_1, lam_2, lam_3)
        security_speed_tradeoff = cp.quad_form(w, Q)
        resource_term = resource_cost(cpu_usages, mem_usages, w, gamma)
        l1_term = alpha * cp.norm1(w)

        #base constraints
        base_constraints = [
            cp.sum(w) == 1,
            w >= 0,
            w <= 1,
            enc_strengths @ w >= 0.0
        ]

        #traffic-type-specific constraints
        if data_type == 'VoIP':
            latency = normalized['latency'].values
            jitter = normalized['jitter'].values
            base_constraints += [
                latency @ w <= 50, #average time for a packet to reach destination. low for speech flow
                jitter @ w <= 5 #variation in delay want consistent to prevent voice distortion.
            ]
        elif data_type == 'Video':
            throughput = normalized['throughput'].values
            error_rate = normalized['error_rate'].values
            base_constraints += [
                throughput @ w >= 400, #how much data can be sent per second
                error_rate @ w <= 1.5 #percentage of corrupt packets lower to preserve video quality
            ]
        elif data_type == 'Cloud':
            packet_loss = normalized['packet_loss'].values #data loss
            base_constraints += [
                packet_loss @ w <= 2.0, #packets never arrive issues with file integrity/ data loss
            ]

        #solve objective
        #obj = Min(w^TQw + c^Tw)
        obj = cp.Minimize(security_speed_tradeoff + resource_term + l1_term)
        prob = cp.Problem(obj, base_constraints)
        sol = prob.solve(solver=cp.SCS)
        print(f"Minimum Cost: {sol:.6f}" if sol else "No solution")

        #print results
        if sol is not None and np.isfinite(sol):
            w_values = np.clip(w.value, 0, 1)
            top_idxs = np.argsort(w_values)[-3:][::-1]

            print(f"\nTop 3 Configurations ({group_name}):")
            print(f"{'Index':<10}{'Weight':>10}{'Enc Time':>12}{'Strength':>12}{'Key Size':>10}{'Packet Size':>15}{'CPU Usage':>12}{'Memory Usage':>15}")

            unnorm_values = pd.DataFrame(scaler.inverse_transform(normalized_part),columns=selected_norm_cols)
            for idx in top_idxs:
                row = unnorm_values.iloc[idx]
                raw_row = raw_subset.iloc[idx]
                orig_row = group_df.iloc[idx]
                #maybe unnormalize the values for print out exclusively
                print(f"{idx:<10}{w_values[idx]:>10.6f}{row['encryption_time']:>12.2f}"
                      f"{row['encryption_strength']:>12.2f}{orig_row['key_size']:>10}"
                      f"{raw_row['packet_size']:>15}{row['cpu_usage']:>12.2f}{row['memory_usage']:>15.2f}")
        else:
            print("No solution for this group.")

In [23]:
#w^TQw, where Q = diag(T_i + lam_1 * (1 - S_i) + lam_2 * E_i - lam_3 * I_i)
#T_i encryption time, S_i encryption strength, E_i error rate, I_i integrity check

# lam_1 weight of encryption strength pen
# lam_2 weight of error rate pen
# lam_3 weight of integrity reward
# alpha weight of L1 spread of the w data
# gamma weight of resource cost
# input a constraint can add if need an extra constraint
# enc_data, data_type=None, alpha=1.0, gamma=1.0, lam_1=1.0, lam_2=1.0, lam_3=1.0
solve_and_report(enc_data)


Solving for Packet Size Group: Small
Minimum Cost: 1.003538

Top 3 Configurations (Small):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
452         1.000000        1.08        9.00     256.0          384.0       10.21         102.03
552         0.000000        5.31       10.00     256.0          402.0       12.83         289.19
342         0.000000        0.66       10.00     256.0          227.0       27.05         130.56

Solving for Packet Size Group: Medium
Minimum Cost: 1.081887

Top 3 Configurations (Medium):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
203         0.561602        3.42        9.00     128.0          798.0       10.81         131.62
16          0.137127        4.68        5.00     192.0          700.0       11.20         127.75
602         0.083571        0.69        7.00     128.0          792.0       13.21         163.74

Solving for Packet Size Group: Large


In [24]:
#voip solve and results
solve_and_report(voip_data, data_type='VoIP')


Solving for Packet Size Group: Small
Minimum Cost: 1.341317

Top 3 Configurations (Small):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
198         0.359546        2.31        8.00     192.0          142.0       10.72         320.32
186         0.174227        4.56        9.00     128.0          202.0       18.81         292.89
135         0.081324        8.31        2.00     256.0          175.0       23.74         139.63

Solving for Packet Size Group: Medium
Minimum Cost: 1.116604

Top 3 Configurations (Medium):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
203         0.514394        3.42        9.00     128.0          798.0       10.81         131.62
147         0.168072        4.01        3.00     128.0          983.0       14.15         100.16
245         0.130722        2.84        6.00     256.0          639.0       11.00         236.98

Solving for Packet Size Group: Large


In [25]:
#vid solve and results
solve_and_report(vid_data, data_type='Video')


Solving for Packet Size Group: Small
Minimum Cost: 1.064385

Top 3 Configurations (Small):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
187         0.753384        1.08        9.00     256.0          384.0       10.21         102.03
213         0.078676        8.68        3.00     192.0          500.0       12.41         219.31
5           0.057532        5.28        5.00     256.0          242.0       17.42         203.36

Solving for Packet Size Group: Medium
Minimum Cost: 1.107765

Top 3 Configurations (Medium):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
232         0.978582        0.69        7.00     128.0          792.0       13.21         163.74
184         0.022299        7.83        1.00     192.0          656.0       11.61         139.40
273         0.000025        0.72        2.00     256.0          854.0       60.71         146.22

Solving for Packet Size Group: Large


In [26]:
#cloud solve and results
solve_and_report(cloud_data, data_type='Cloud')


Solving for Packet Size Group: Small
Minimum Cost: 1.247910

Top 3 Configurations (Small):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
155         0.508491        3.61        4.00     256.0          290.0       25.00         116.82
15          0.158633        0.93        9.00     192.0          483.0       21.49         274.43
116         0.093704        2.63        4.00     128.0          343.0       23.50         122.27

Solving for Packet Size Group: Medium
Minimum Cost: 1.110535

Top 3 Configurations (Medium):
Index         Weight    Enc Time    Strength  Key Size    Packet Size   CPU Usage   Memory Usage
139         0.805660        4.84       10.00     256.0          785.0       11.45         194.01
85          0.085251        4.20       10.00     128.0          869.0       14.56         119.27
313         0.032241        4.40        3.00     256.0          575.0       10.44         136.18

Solving for Packet Size Group: Large
